# Preworkout

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications import vgg16
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/datasets/unbalanced_dataset_2304.zip

# Model

We use the VGG16 model, pre-trained on imagenet.

VGG16 is a deep network with 13 convolutional layers. It was previously trained on millions of images, and has over 100,000,000 weights and biases, the majority of which connect to the first fully-connected layer (fc1). 

VGG-16 is setup to take a fixed-size (224 x 224 x 3) RGB image at its input, and then forward it through a series of altrnating convolutional and max-pooling layers, then capped off by three fully-connected layers of 4096, 4096, and 1000 neurons, where the last layer is our softmax classification layer.

Notice that the output shape at each layer has `None` the first dimension. This is because the network can process multiple images in a single batch. So if you forward 5 images at shape [5, 224, 224, 3], then the output shape at each layer will be 5 in the first dimension.

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=True)

553476096/553467096 [==============================] - 3s 0us/step


## Feature extraction

What we have in the model variable is a highly effective image classifier trained on the ImageNet database. We expect that the classifier must form a very effective representation of the image in order to be able to classify it with such high accuracy. We can use this to our advantage by re-purposing this for our image retrieval task. Therefore we copy the model, but remove the last layer (the classification layer), so that the final layer of the new network, called feat_extractor is the second 4096-neuron fully-connected layer, "fc2 (Dense)".

In [ ]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)


# Evaluate

## test dataset

We now create the variables query_images and gallery_images, which are lists of paths of the various pictures.
Then, we will begin a loop which will open each image, extract its feature vector, and append it to two lists called query_features and gallery_features which will contain our activations for each image.

In [ ]:
import utils
GALLERY_PATH = "/content/gallery/"
QUERY_PATH = "/content/query/"

target_shape = (224, 224)

gallery = utils.Dataset(GALLERY_PATH, target_shape = target_shape).get_dataset()
query = utils.Dataset(QUERY_PATH, target_shape = target_shape).get_dataset()



Loaded 8777 images from /content/unbalanced_dataset_2304/validation/gallery/ 
Loaded 2236 images from /content/unbalanced_dataset_2304/validation/query/ 


## extract features

In [ ]:
Now we use our feature extractor on our competition data. 
The compute_features function returns an array with one element per image. Each element contains a 4096-element array, 
which is the activations of the last fully-connected layer fc2 in VGG16.

We expect that the fc2 activations form a very good representation of the image, such that similar images should 
produce similar activations. In other words, the fc2 activations of two images which have similar content should 
be very close to each other. We can exploit this to do information retrieval. 

In [ ]:
query_features, query_urls, query_labels = utils.compute_features(query, feat_extractor)


In [ ]:
gallery_features, gallery_urls, gallery_labels = utils.compute_features(gallery, feat_extractor)


## without PCA

We are now ready to do our reverse image queries. Both query_features and gallery_features contain a compact representation of our images, one 4096-element row for each image. The assumption we can now make is that two images which have similar content, should produce similar feature vectors.
In order to do image retrieval, though, we first need to decide a measurement of the distance between each query feature vector and all the gallery ones. We choose to use both euclidean distance and cosine similarity.

In [ ]:
results_no_PCA_eu = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls)
results_no_PCA_cos = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls, dist= 'cosine')

## with PCA

So, we are done with our image retrieval task. Nonetheless, we want to try also to do something more on top of this.
In particular, we would like to apply a PCA algorithm on the 4096-element feature vector to try to reduce the dimensionality of our feature vectors down to 30. This is for two reasons: 
1) the 4096 feature vector may have some redundancy in it, such that multiple elements in the vector are highly correlated or similar. This would skew similarity comparisons towards those over-represented features. 
2) Operating over 4096 elements is inefficient both in terms of space/memory requirements and processor speed, and it would be better for us if we can reduce the length of these vectors but maintain the same effective representation.
3) We are also interested in seeing how much our retrieval performance decreases when we go from a 4096-element vector to a 30-element one. As a matter of fact, we are decreasing our vector size by a factor of 128!

The next cell will instantiate a PCA object, which we will then fit our data to, choosing to keep the top 30 principal components.

In [ ]:
pca = PCA(n_components=30)
pca.fit(query_features)

pca = PCA(n_components=30)
pca.fit(gallery_features)


The pca object stores the actual transformation matrix which was fit in the previous cell. We can now use it to transform any original feature vector (of length 4096) into a reduced 30-dimensional feature vector in the principal component space found by the PCA. 

So we take our original feature vectors, and transform them to the new space.

In [ ]:
query_pca_features = pca.transform(query_features)
gallery_pca_features = pca.transform(gallery_features)

In [ ]:
results_PCA = utils.compute_results(query_pca_features, gallery_pca_features, query_urls, gallery_urls)

# competition

In [ ]:
import requests
import json

def submit(results, url="http://tinyurl.com/IML2022"):
    res = json.dumps(results)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['results']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")


In [ ]:

mydata = dict()
mydata['groupname'] = "The Ostriches"

mydata["images"] = results_PCA
submit(mydata)

mydata["images"] = results_no_PCA_eu
submit(mydata)

mydata["images"] = results_no_PCA_cos
submit(mydata)

## compute results 

In [ ]:
utils.evaluate(results_PCA, query_labels, gallery_labels)
utils.evaluate(results_no_PCA_eu, query_labels, gallery_labels)
utils.evaluate(results_no_PCA_cos, query_labels, gallery_labels)